In [21]:
import pandas as pd
import sklearn
import numpy as np

data = pd.read_excel('../Adatok/tisztitott_adat.xlsx', header=0,index_col=0)
data

,date,text,president,label
0,2012-11-05,president obama tells the story of fired up re...,Obama,0
1,2012-11-06,election day is here confirm your polling plac...,Obama,1
2,2012-11-07,thank you president obama in his victory speec...,Obama,2
3,2012-11-08,the definition of hope is you still believe ev...,Obama,2
4,2012-11-09,what bobby kennedy called the ripples of hope ...,Obama,0
...,...,...,...,...
1823,2020-06-11,our great national guard troops who took care ...,Trump,2
1824,2020-06-12,people have no idea how fake the lamestream me...,Trump,2
1825,2020-06-15,i’ve done more in less than 4 years than biden...,Trump,1
1826,2020-06-16,wow may retail sales show biggest one-month in...,Trump,0


In [22]:
import gensim
tokenized_tweet = data['text'].apply(lambda x: x.split())

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=50, 
            window=5, 
            min_count=2,                                   
            sg = 1,
            hs = 0,
            negative = 10, 
            workers= 32, 
            seed = 34
)

model_w2v.train(tokenized_tweet, total_examples= len(data['text']), epochs=20)

(4997854, 6534600)

In [23]:
model_w2v.wv.most_similar(positive="trump")

[('donald', 0.873864471912384),
 ('sabotage', 0.7372989654541016),
 ('j', 0.7238896489143372),
 ('media’s', 0.7036998271942139),
 ('campaign', 0.6925650835037231),
 ('foundation', 0.6880016922950745),
 ('trump’s', 0.6835409998893738),
 ('clinton', 0.676448404788971),
 ('i’d', 0.6709875464439392),
 ('scheme', 0.6678166389465332)]

In [24]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [25]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 50)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 50)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(1828, 50)

In [26]:
vec_data = pd.concat([wordvec_df, data['label']], axis=1)
vec_data

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,label
0,0.176195,0.400107,0.063833,-0.047243,0.379398,0.001906,-0.240907,-0.013293,-0.253030,-0.188061,...,-0.611825,-0.092432,0.026429,0.117207,-0.073067,0.311092,-0.450759,0.145084,-0.355200,0
1,0.308915,0.387523,-0.073404,-0.127846,0.463324,-0.039386,-0.259344,-0.122964,-0.301306,-0.129755,...,-0.615937,-0.036920,0.125880,-0.037207,-0.190333,0.163252,-0.602922,0.143013,-0.436464,1
2,0.276564,0.337364,0.037026,-0.077074,0.371610,0.086894,-0.138589,-0.067244,-0.186002,-0.070232,...,-0.574181,-0.063649,0.189423,-0.030094,-0.066840,0.109191,-0.605567,0.172048,-0.408675,2
3,0.231620,0.360612,-0.085521,-0.126497,0.320687,-0.128666,-0.109749,-0.170012,-0.286664,-0.001804,...,-0.711585,-0.130762,0.212588,0.027735,-0.103147,0.099261,-0.483103,0.304043,-0.399509,2
4,0.189428,0.387376,-0.192068,-0.091749,0.427815,-0.127749,-0.082616,-0.118818,-0.252675,-0.138974,...,-0.610949,-0.067066,0.220233,0.000427,-0.137080,0.185548,-0.602420,0.219906,-0.541185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,0.231262,0.357693,-0.189297,-0.123807,0.293092,-0.016057,-0.132901,-0.048584,-0.239733,0.005645,...,-0.592727,-0.181990,0.205931,-0.030121,-0.198894,0.113111,-0.566411,0.008201,-0.389829,2
1824,0.293496,0.337836,-0.118748,-0.110966,0.267164,-0.084375,-0.154838,-0.106967,-0.210433,-0.026474,...,-0.628223,-0.159698,0.185550,0.017761,-0.151750,0.154438,-0.527447,0.047161,-0.401636,2
1825,0.312147,0.410976,-0.177270,-0.093560,0.151374,-0.159984,-0.165689,-0.121895,-0.261683,-0.004379,...,-0.595919,-0.163216,0.128714,-0.064329,-0.178822,0.122368,-0.468430,0.094918,-0.391055,1
1826,0.190224,0.292260,-0.244064,-0.202552,0.327836,-0.042354,-0.148880,-0.051078,-0.193633,0.066048,...,-0.587513,-0.212488,0.241919,0.018754,-0.178455,0.085308,-0.580205,-0.022784,-0.385212,0


In [27]:
from sklearn.model_selection import train_test_split

label = vec_data['label']
attrs = vec_data.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(
        attrs, label, test_size=0.25, random_state=1000)

In [28]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5842450765864332


In [29]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=30, random_state=0)
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.5645514223194749
